# __Implementasi Reinforcement Learning__

Kelompok Terserah :

| Nama                       | NRP        |
| -------------------------- | ---------- |
| Muhammad Abrol AL Qushoyyi | 5025201028 |
| Moh. Ilham Fakhri Zamzami  | 5025201275 |
| Arief Badrus Sholeh        | 5025201228 |
| Ahmad Hafizh Assa'ad       | 5025201106 |

#### __STUDI KASUS : Self-Driving Cab__

https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

Tugas _Self-Driving Cab_ adalah menjemput penumpang di satu lokasi dan menurunkannya di lokasi lain. Berikut adalah beberapa hal yang _Self-Driving Cab_ dapat ditangani:

- Menurunkan penumpang ke lokasi yang tepat.
- Menghemat waktu penumpang dengan mengambil waktu seminimal mungkin untuk mengantar
- Menjaga keselamatan penumpang dengan mematuhi aturan lalu lintas

__Reward__

- Agent menerima reward positif yang tinggi apabila penurunan berhasil
- Agent menerima reward negatif apabila menurunkan penumpang di lokasi yang salah
- Agent menerima __sedikit__ reward negatif karena tidak sampai ke tujuan setelah setiap langkah-waktu. "Sedikit" reward negatif karena lebih baek agent tiba terlambat daripada melakukan langkah yang salah dengan mencoba mencapai tujuan secepat mungkin

__State__

- Terdapat satu taksi pada tempat parkir
- Tempat parkir berupa 5x5 grid, yang memberikan 25 kemungkinan lokasi taksi
- Terdapat 4 lokasi untuk menjemput dan menurunkan penumpang. R, G, Y, B dengan koordinat [(0,0), (0,4), (4.0), (4,3)]
- Perhitungkan juga 1 state tambahan untuk penumpang berada di dalam taksi
- Sehingga terdapat 4 lokasi tujuan dan (4+1) lokasi penumpang.
- Total state pada Environment = 5 x 5 x 5 x 4 = 500 Total kemungkinan state.

__Action__

Terdapat 6 kemungkingan tindakan:
- `south`
- `north`
- `east`
- `west`
- `pickup`
- `dropoff`

#### __Implementasi menggunakan _Python___

Menggunakan bantuan _OpenAI Gym Environment_ yaitu `Taxi-V3`



In [6]:
import gym

env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



- Kotak yang terisi melambangkan taksi, yang berwarna kuning tanpa penumpang dan hijau dengan penumpang.
- Simbol ("|") mewakili dinding yang tidak dapat dilintasi taksi.
- R, G, Y, B adalah kemungkinan lokasi penjemputan dan tujuan. Huruf biru mewakili lokasi penjemputan penumpang saat ini, dan huruf ungu adalah tujuan saat ini.

__Problem Statement__

> _Ada 4 lokasi (diberi label dengan huruf berbeda), dan tugas agent adalah menjemput penumpang di satu lokasi dan menurunkannya di lokasi lain. agent menerima +20 poin untuk penurunan yang berhasil dan kehilangan 1 poin untuk setiap kali- langkah yang diperlukan. Ada juga penalti 10 poin untuk tindakan penjemputan dan pengantaran ilegal._

In [15]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


__Reward Table__

Ketika Environment Taksi dibuat, reward table 'P' juga diinsisasi. Direpresentasikan dengan matriks yang memiliki jumlah state sebagai baris dan jumlah action sebagai kolom.

In [17]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

> Struktur matrix ```{action: [(probability, nextstate, reward, done)]}.```

__Ilustrasi__

Dengan menggunakan encoding method, kita bisa melakukan

In [18]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



`State=328` digunakan sebagai ilustration state yang digunakan

#### __Penyelesaian tanpa Reinforcement Learning__

In [28]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 537
Penalties incurred: 175


In [30]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 537
State: 0
Action: 5
Reward: 20


#### __Penyelesaian dengan Reinforcement Learning__

menggunakan estimasi Q-Learning untuk memaksimalkan __*Future Reward*__

Pertama, inisialisasi Q-Table dengan 500 x 6 matriks null

In [31]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

__Training Agent__

In [32]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

Wall time: 49.8 s


In [33]:
q_table[328]

array([ -2.4058189 ,  -2.27325184,  -2.41321436,  -2.36109819,
       -10.96279873, -10.86407837])

__Testing Agent__

In [34]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 12
Penalties incurred: 0


In [38]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.3)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 12
State: 0
Action: 5
Reward: 20


__Evaluating Agent__

In [39]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.29
Average penalties per episode: 0.0
